## Web-Scraper


In [1]:

from random import randint
from time import time 
import requests
from requests import get
from warnings import warn 
from bs4 import BeautifulSoup
from IPython.core.display import clear_output
from time import sleep
from selenium import webdriver
import csv 

# set errors to track number of erros
errors = 0

# track time during web scraping
start_time = time()

# track number of requests
requests = 0

# go through pages of craigslist rentals, divided 120 each page
# and seemingly 2400 or less listings in total for the default Minneapolis
# craigslist page
listings_pages = [str(i) for i in range(0, 2400) if (i%120==0)]




# iterate through listing pages which 
# contain numbers between 0-2400 divisble by 120
for page in listings_pages:
     
    # for page increment value by 120 each time to loop through 
    # groupings size 120 of craigslist rental ads and make request to url plus
    # grouping indicated by number divisible by 120
    driver = get('https://minneapolis.craigslist.org/search/apa?s=' + (page))
    
    
    # create beautiful soup object
    listing_object = BeautifulSoup(driver.content, 'html.parser')
    

    # listing containers contains all 120 urls from the url requested with driver variable

    listing_containers = listing_object.find_all('li', class_="result-row")

    # for each individual url contained in listing_containers
    for item in listing_containers:
        # increment requests for tracking purposes
        requests += 1 
        # track time
        elapsed_time = time() - start_time
        print ('Request: {}; Frequency: {} request/s'.format(requests, requests/elapsed_time))
        # clear output for continous display of values
        clear_output(wait = True)

        # stop loop at 2500 requests
        if requests > 2500:
            warn('Number of requests was greater than 2280')
            break
        try:
            url = item.find('a', class_="result-title hdrlnk")['href']
            
        except AttributeError:
            print ('cant append')
            
            
       

        
        try:
            # use requests to make request to url leading to single rental listing
            link = get(url)
            
            # create Beatufiul Soup object
            listing_object = BeautifulSoup(link.content, 'html.parser')
            
            # find everything in the "section tag of each url
            page_items = listing_object.find_all('section', class_="page-container")

            # scrape desired attributes by looping through 
            # "section" tag of each url
            for container in page_items:
                # these lists will become the columns for the data frame 
                # columns that they become are commented above each respective list
                # the url list is not here but the variable is already set during each iteration as the 
                
                # 'price'
                prices = []
                # 'beds_baths'
                bed_baths = []
                # 'address'
                addrs = []
                # 'latitude'
                lats = []
                # 'longitude'
                lons = []
                # 'map accuracy'
                accuracy = []
                # 'datetime'
                date_times = []
                # 'title'
                data_titles = []
                # 'post id'
                posting_ids = []
                # 'square feet'
                areas = []
                # 'square feet 2'
                title_price_and_attributes = []
                

               
                # get price
                try:
                    price = container.h2.find("span", class_="price").text
                    prices.append(price)
                except AttributeError:
                    prices.append("N/A")
                except TypeError:
                    prices.append("N/A")

                       # get latitutude
                try:
                    lat = container.find('div', class_="viewposting")['data-latitude']
                    lats.append(lat)
                except TypeError:
                    lats.append("N/A")
                except AttributeError:
                    lats.append("N/A")

                        # get longitude
                try:
                    lon = container.find('div', class_="viewposting")['data-longitude']
                    lons.append(lon)
                except TypeError:
                    lons.append("N/A")
                except AttributeError:
                    lons.append("N/A")

                        # get map data accuracy
                try:
                    data_accuracy = container.find('div', class_="viewposting")['data-accuracy']
                    accuracy.append(data_accuracy)
                except TypeError:
                    accuracy.append("N/A")
                except AttributeError:
                    accuracy.append("N/A")





                        # get datetime
                try:
                    date_time = container.find('time', class_="date timeago")['datetime']
                    date_times.append(date_time)
                except TypeError:
                    date_times.append("N/A")
                except AttributeError:
                    date_times.append("N/A")

                    # get bedrooms and baths
                try:
                    bed_bath = container.section.find_all("span", class_="shared-line-bubble")[0].text
                    bed_baths.append(bed_bath)
                except AttributeError:
                    bed_baths.append("N/A")
                except TypeError:
                    bed_baths.append("N/A")
                except IndexError:
                    bed_baths.append("N/A")

                # get square feet

                try: 
                    sqrft = container.section.find_all("span", class_="shared-line-bubble")[1].text
                    areas.append(sqrft)
                except AttributeError:
                    areas.append("N/A")
                except TypeError:
                    areas.append("N/A")
                except IndexError:
                    areas.append("N/A")



                # get address 
                try:   
                    addr = container.section.section.find("div", class_="mapaddress").text
                    addrs.append(addr)
                except AttributeError:
                    addrs.append("N/A")

                 # get title
                try:
                    data_title = container.h2.find(id="titletextonly").text
                    data_titles.append(data_title)
                except AttributeError:
                    data_titles.append("N/A")
                    
                
                    # get posting id
                try:
                    posting_id = container.find_all('p', class_="postinginfo")[1].text
                    posting_ids.append(posting_id)
                except AttributeError:
                    posting_ids.append("N/A")
                except IndexError:
                    posting_ids.append("N/A")
                
                
                 # this will be information in square feet 2
                try:
                    price_and_attributes = container.h2.find('span', class_="housing").text
                    title_price_and_attributes.append(price_and_attributes)
                except AttributeError:
                    title_price_and_attributes.append("N/A")
                
               

                # dataframe is updated in each loop 



                apartment_data = pd.DataFrame({'Url' : url, 
                                           'Price': prices,
                                           'Address': addrs,
                                           'Latitude': lats,
                                           'Longitude': lons,
                                           'Map Accuracy': accuracy,
                                           'Date & Time Stamp': date_times,
                                           'Bath/Bed': bed_baths, 
                                           'Areas' : areas,
                                           'Listing Title': data_titles,
                                           'ids' : posting_ids,
                                           'Header': title_price_and_attributes
                                           })
                # leave dataframe open and call "a" for append option to append values iteratively
                with open("Craigslist_ApartmentData_Minneapolis_4_17.csv", "a", encoding='utf-8') as f:
                    apartment_data.to_csv(f, header=False)
        # keep track of requests that come back errors
        except:
            errors += 1
            print(errors)
            continue
    

2400


# PREPARE SCRAPED DATA FOR GEOCODING


In [ ]:
# pandas to work with dataframes
import pandas as pd
# import re to use regular expressions
import re
# import numpy
import numpy as np
# import glob to bring together multiple csv files
import glob

# search for csvs containing web-scraped data with the name scheme
csvs = glob.glob(r'*Craigslist_ApartmentData_Minneapolis*.csv')

# set dfs list to store each dataframe that gets read in
dfs = []

# iterate through csv files gathered in glob
for file in csvs:
    # read each csv as a dataframe with the same column names
    df1 = pd.read_csv(file, names=['address', 'square feet', 'beds_baths', 'datetime', 'square feet 2', 'latitude', 'title', 'longitude', 'map accuracy', 'price', 'url', 'post id'])
    # append dataframe to dfs list
    dfs.append(df1)

# concatenate dataframes in dfs together vertically
df = pd.concat(dfs, axis=0)
# check to see if address present, if it is, the boolean will be set to False
# this seems counterintuitive
df['address_check'] = [bool(re.match('nan', str(cell))) for cell in df['address']]
# sort values by address_check column
df = df.sort_values(['address_check'])
# set an index of integers equal to the length of dataframe rows
index_list = list(range(len(df.index)))
# make index series a column in dataframe
df['index'] = index_list
# set index column as index of dataframe
df = df.set_index(['index'])

# set entire df type as string, as most operations will require data type to be string
df = df.astype(str)

# drop any post id values that are null
postid_no_null = df.dropna(subset = ['post id'])



# create dataframe where "post id" column is null
null_id = df[df['post id'].isnull()]
# drop duplicates in null_id dataframe using the "title" column
null_id = null_id.drop_duplicates(subset=['title'], keep='first')
# drop duplicates using "post id" column in the dataframe containing no null "post id" values
postid_no_null = df.drop_duplicates(subset=['post id'], keep='first')
# concatenate vertically null_id and postid_no_null dataframes
df = pd.concat([null_id, postid_no_null])
# drop latitude, longitude, map accuracy columns 
df = df.drop(columns=['latitude', 'longitude', 'map accuracy'])


# extract and store all possible valid square feet or beds values across columns where they might be
# using findall function

# column to store values of square feet that exist in column
df['value_square_feet_two'] = [re.findall('\d+ft', str(cell)) for cell in df['square feet 2']]
# create column of all patterns of square feet that exist in  beds_baths column
df['value_square_feet_three'] = [re.findall('\d+ft', str(cell)) for cell in df['beds_baths']]
# get all patterns of beds that exist in "square feet 2" column
df['number_of_beds_2'] = [re.findall('\d+br', str(cell)) for cell in df['square feet 2']]
# clean values from columns to prepare them to be brought into "square feet", "price" and "beds_baths" columns
df['number_of_beds_2'] = df['number_of_beds_2'].astype(str)
df['value_square_feet_two'] = df['value_square_feet_two'].astype(str)
df['value_square_feet_three'] = df['value_square_feet_three'].astype(str)

# create a boolean of whether or not a beds/bath pattern exists in the "beds_baths" column
df['first_beds_boolean'] = [bool(re.search('\d+[Bb]', cell)) for cell in df['beds_baths']] 
# get boolean of whether or not beds value is present in "square feet 2" column
df['second_beds_boolean'] = [bool(re.search('\d+br', str(cell))) for cell in df['square feet 2']]
# clean value before moving
df.loc[df['second_beds_boolean'] == 1, 'number_of_beds_2'] = df['number_of_beds_2'].apply(lambda x: str(x[2:-2]))
# where value in "beds_baths" is not available, and is available in "square feet 2", 
# fill with the beds value of "square feet 2", stored in "number_of_beds_2"
df.loc[(df['second_beds_boolean'] == 1) & (df['first_beds_boolean'] == 0), 'beds_baths'] = df['number_of_beds_2']
# create a boolean of whether or not a square feet pattern exists in the "square feet" column
df['first_square_feet_boolean'] = [bool(re.search('\d+ft', str(cell))) for cell in df['square feet']]
# boolean to see if row in square feet 2 column contains square feet value
df['second_square_feet_boolean'] = [bool(re.search('\d+ft', str(cell))) for cell in df['square feet 2']]
# clean value before moving
df.loc[df['second_square_feet_boolean'] == 1, 'value_square_feet_two'] = df['value_square_feet_two'].apply(lambda x: str(x[2:-2]))
# where value in "square feet" is not available, and is available in "square feet 2", 
# fill with the square feet value of "square feet 2", stored in "value_square_feet_two"
df.loc[(df['first_square_feet_boolean'] == 0) & (df['second_square_feet_boolean'] == 1), 'square feet'] = df['value_square_feet_two']
# boolean to see if beds_baths column is True/False for square feet pattern
df['third_square_feet_boolean'] = [bool(re.search('\d+ft', str(cell))) for cell in df['beds_baths']]
# clean value before moving
df.loc[df['third_square_feet_boolean'] == 1, 'value_square_feet_three'] = df['value_square_feet_three'].apply(lambda x: str(x[2:-2]))
# for all values in beds_bath where the square feet pattern is true, 
# cell value is updated with number in square_feet_one
df.loc[(df['first_square_feet_boolean'] == 0) & (df['second_square_feet_boolean'] == 0) & (df['third_square_feet_boolean'] == 1), 'square feet'] = df['value_square_feet_three']
# for square feet, beds_baths and price columns, determine if appropriate values are present with boolean
# in order to fill these columns with appropriate values in future operations
df['square feet boolean'] = [bool(re.search('\d+ft', str(cell))) for cell in df['square feet']]
df['beds boolean'] = [bool(re.search('\d+[Bb]', str(cell))) for cell in df['beds_baths']]
df['price boolean'] = [bool(re.search('\d+', str(cell))) for cell in df['price']]


# get just digits in square feet column 
df.loc[(df['square feet boolean'] == 1), 'square feet'] = df['square feet'].apply(lambda x: x[0:-3])
# get just numerical value of beds_baths
df.loc[df['beds_baths'].str.contains(r'B'), 'beds_baths'] = df['beds_baths'].apply(lambda x: x[0])
# get just price in "price" column
df['price'] = df.price.apply(lambda x: x.replace('$', ''))
# identify and drop posts associated with the superbowl
df['check_superbowl'] = [bool(re.search("SUPERBOWL|Superbowl|SUPER BOWL|Super bowl|superbowl|super bowl|Super Bowl", cell)) for cell in df['title']]
df = df.drop(df[df['check_superbowl'] == 1].index)
# extract url information from "url" column and fill as value in "county" column
df['county'] = [str(re.findall('org/.../', cell))[6:9] for cell in df['url']]


# get latitude and longitude of counties and assign to respective "county_lat" and "county_lon" coordinates

# create county_lat, county_lon columns
df['county_state'] = df['county']
df['county_state'] = df['county']



# create a column county_state with county and state to append to each respective address
df['county_state'] = df['county']
df.loc[(df['county'] == "hnp"), "county_state"] = " ,Hennepin County, Minnesota"
df.loc[(df['county'] == "ram"), "county_state"] = " ,Ramsey County, Minnesota"
df.loc[(df['county'] == "dak"), "county_state"] = " ,Dakota County, Minnesota"
df.loc[(df['county'] == "ank"), "county_state"] = " ,Anoka County, Minnesota"
df.loc[(df['county'] == "wsh"), "county_state"] = " ,Washington County, Minnesota"
df.loc[(df['county'] == "csw"), "county_state"] = " ,Carver County, Minnesota"
# append the value in county_state to each respective address
df.loc[(df['address_check'] == True), 'address'] = df[['address', 'county_state']].apply(''.join, axis=1)
# create geocode_lat and geocode_lon columns for geocoding
df['geocode_lat'] = "none"
df['geocode_lon'] = "none"

# save dataframe without missing addresses to be geocoded
# then resume script after values have been geocoded

df.to_csv('geocode_craigslist_rentals_2.csv')

## Geocoding Script

In [ ]:
import googlemaps
from datetime import datetime
import pandas as pd
from IPython.core.display import clear_output
from random import randint
from time import time 
import requests
from requests import get
from warnings import warn 
from bs4 import BeautifulSoup
from IPython.core.display import clear_output
from time import sleep

from itertools import islice
csv = "geocode_craigslist_rentals_2.csv"

  

df = pd.read_csv(csv)

index_list = list(range(len(df.index)))
df['index'] = index_list

df = df.set_index(['index'])



count = 0

for index, row in islice(df.iterrows(),5000, 7500):
    if (row['geocode_lat'] == "none") & (row['geocode_lon'] == "none") & (row['address_check'] == False):
        

    

        try:
            key = "string given by google"
            sleep((2))
            gmaps = googlemaps.Client(key= key)
            address = str(row['address'])
            geocode_result = gmaps.geocode(address)
            latitude = geocode_result[0]['geometry']['location']['lat']
            longitude = geocode_result[0]['geometry']['location']['lng'] 
            df.loc[index, "geocode_lon"] = str(longitude)
            df.loc[index, "geocode_lat"] = str(latitude)
            if (count %50 ==0):
                print (row['geocode_lon'], row['address'])
            count += 1
            print ('Request: {}'.format(count))
            clear_output(wait = True)


        except:
            if (count %50 ==0):
                print (row['geocode_lon'], row['address'])

            df.loc[index, "geocode_lon"] = "error"
            df.loc[index, "geocode_lat"] = "error"


            count += 1
            print ('Request: {}'.format(count))
            clear_output(wait = True)


       


    
df.to_csv('geocode_craigslist_rentals_2.csv', header=True)

 ## Bring Data Together After Geocoding

In [15]:
# pandas to work with dataframes
import pandas as pd
# import re to use regular expressions
import re
# import numpy
import numpy as np
# import glob to bring together multiple csv files
import glob

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import bokeh.plotting as bkp
from mpl_toolkits.axes_grid1 import make_axes_locatable


# search for csvs containing web-scraped data with the name scheme
csvs = glob.glob(r'*geocode_craigslist*.csv')

# set dfs list to store each dataframe that gets read in
dfs = []

# iterate through csv files gathered in glob
for file in csvs:
    # read each csv as a dataframe with the same column names
    df1 = pd.read_csv(file)
    # append dataframe to dfs list
    dfs.append(df1)

# concatenate dataframes in dfs together vertically
df = pd.concat(dfs, axis=0)
# set an index of integers equal to the length of dataframe rows
index_list = list(range(len(df.index)))
# make index series a column in dataframe
df['index'] = index_list
# set index column as index of dataframe
df = df.set_index(['index'])

# set entire df type as string, as most operations will require data type to be string
df = df.astype(str)


# get only rows from ramsey and hennepin county, and only rows where geocode_lat and geocode_lon are not "none
df_clean = df.loc[(df['geocode_lat'] != "none") & (df['geocode_lon'] != "none")]
df_clean = df_clean.loc[(df_clean['county'] == "ram") | (df_clean['county'] == "hnp")]
# remove "geocode_lat" and "geocode_lon" columns with "error" value
df_clean = df_clean.loc[(df_clean['geocode_lat'] != "error") & (df_clean['geocode_lon'] != "error")]
# drop columns that will not be necessary for analysis 
df_clean = df_clean.drop(["county_state", "number_of_beds_1", "latitude", "longitude", "map accuracy", "square_feet_three", "square_feet_two", "square_feet_one", "Unnamed: 0", "address_check", "address_county", "address_lat", "address_lon", "check_superbowl", "county_lat", "county_lon", "square feet 2", "value_square_feet_two", "value_square_feet_three", "number_of_beds_2", "first_beds_boolean", "second_beds_boolean", "first_square_feet_boolean", "second_square_feet_boolean", "third_square_feet_boolean"], axis = 1)    
df_clean = df_clean.astype(str)


In [16]:
# subset dataset to only contain geocoded values in Ramsey and Hennepin County
import geopandas as gpd
from shapely.geometry import Point
counties_data = "mn_county_boundaries_1500.shp"
crs = {'init': 'epsg:4326'}
counties = gpd.read_file(counties_data)
counties = counties.to_crs({'init': 'epsg:4326'})

df_clean['geocode_lon'] = df_clean['geocode_lon'].astype(float)
df_clean['geocode_lat'] = df_clean['geocode_lat'].astype(float)
# convert latitude and longitude points into GIS data points
geometry = [Point(xy) for xy in zip(df_clean['geocode_lon'], df_clean['geocode_lat'])]

locations = gpd.GeoDataFrame(df_clean, crs=crs, geometry=geometry)
locations = locations.to_crs({'init': 'epsg:4326'})

ramsey_hennepin = counties.loc[(counties['CTY_NAME'] == 'Hennepin') | (counties['CTY_NAME'] == 'Ramsey')]

counties_subset = gpd.sjoin(locations, ramsey_hennepin, how="right", op='within')
counties_subset = counties_subset.iloc[:, :16]
# set an index of integers equal to the length of dataframe rows
index_list = list(range(len(counties_subset.index)))
# make index series a column in dataframe
counties_subset['index'] = index_list
# set index column as index of dataframe
counties_subset = counties_subset.set_index(['index'])
counties_subset = pd.DataFrame(counties_subset)

## Final Data Cleaning With Geocoded and Subsetted Data

In [18]:
from collections import Counter


#convert counties_subset beds_baths to string
counties_subset['beds_baths'] = counties_subset['beds_baths'].astype(str)



# create column in clean dataframe to verify if bed value is assigned in code to follow
counties_subset['bed assigned'] = False

# assign 'bed assigned' column to false for values that need to be inspected
counties_subset.loc[(counties_subset['beds_baths'] == "0") | (counties_subset['beds_baths'] == "0.0") | (counties_subset['beds boolean'] == "False"), 'bed assigned'] = False


# take a subset of listings without a beds value, with 0 as a bed value, or 0.0
# as the titles of this subset will be insepcted to see if they can be assigned a beds value
beds_none = counties_subset.loc[(counties_subset['beds_baths'] == "0") | (counties_subset['beds_baths'] == "0.0") | (counties_subset['beds boolean'] == "False")] 


# drop any listing that has beds, price and square feet values as null
beds_none = beds_none.drop(beds_none[(beds_none['beds boolean'] == "False") & (beds_none['price boolean'] == "False") & (beds_none['square feet boolean'] == "False")].index)


# create a list of bed values and their index
beds_none_list = beds_none.index.values.tolist()
# create a list of titles
titles_list = beds_none['title'].values.tolist()
# zip the two lists so that the two values remain together
beds_none_list = list(zip((beds_none_list), titles_list))
# match indexes that do not get assigned
# and create a unique set later
no_match_indices = []
# search for titles with "studio" in the title
pattern = re.compile('studio', re.IGNORECASE)
studios = [value if bool(re.search(pattern, str(value[1]))) is True else no_match_indices.append(value[0]) for value in beds_none_list]
# search for titles that suggest they are 1 bedroom
pattern = re.compile('one bedroom|1 bedroom|1 br|1br', re.IGNORECASE)
ones = [value if bool(re.search(pattern, str(value[1]))) is True else no_match_indices.append(value[0]) for value in beds_none_list]

# search for titles that suggest they are 2 bedroom
pattern = re.compile('two bedroom|2 bedroom|2 br|2br', re.IGNORECASE)
twos = [value if bool(re.search(pattern, str(value[1]))) is True else no_match_indices.append(value[0]) for value in beds_none_list]

# search for titles that suggest they are 3 bedroom
pattern = re.compile('three bedroom|3 bedroom|3 br|3br', re.IGNORECASE)

three = [value if bool(re.search(pattern, str(value[1]))) is True else no_match_indices.append(value[0]) for value in beds_none_list]

# search for titles that suggest they are 4 bedroom
pattern = re.compile('four bedroom|4 bedroom|4 br|4br', re.IGNORECASE)

# search for titles that suggest they are 4 bedroom
four = [value if bool(re.search(pattern, str(value[1]))) is True else no_match_indices.append(value[0]) for value in beds_none_list]

# see if there are any titles that were duplicated across the lists
# to check integrity of regular expression pattern matching operations
beds_number_assigned = ones + studios + twos + three + four

# use a Counter to determine which titles occur more than once in main list of 
# beds_number_assigned
count_beds_number = Counter(beds_number_assigned)
count_beds_number = {k:count_beds_number[k] for k in count_beds_number if count_beds_number[k] > 1}

# a 12 bedroom listing appears frequently, which doesn't really have a place in the analysis 


# take a set of no_match indices so that it is unique
no_match_indices = set(no_match_indices)
# create list of lists of bedroom types that were or were not discovered
bed_assignments_list = [studios, ones, twos, three, four]


# compare list of indices with no match to dataframe 
# to indicate where in dataframe a beds none value was not matched
no_match = counties_subset.index.isin(no_match_indices)
counties_subset['no match'] = no_match
# loop through subset lists of beds values
increment = 0

for item in bed_assignments_list:

    # filter nonetypes from list
    indices = [x for x in item if x != None]
    # get indices of bed values that were assigned using 'titles' column
    # so they can be compared with indices of counties_subset

    indices = [x for x, y in indices]
    
    # compare indices assigned using 'titles' column
    # to indices assigned 
    idx = counties_subset.index.isin(indices)


    # add 'index match' column to counties_subset dataframe
    counties_subset['index match'] = idx


    # where 'index match' is true, set 'bed assigned' to yes, if false assign bed to 

    counties_subset.loc[(counties_subset['index match'] == True), 'bed assigned'] = True

    # for studios group, assign value to studio
    if increment == 0:
        counties_subset.loc[counties_subset['index match'] == True, 'beds_baths'] = "studio"
      
    # for other groups, assign to number of beds
    else:
    
        counties_subset.loc[counties_subset['index match'] == True, 'beds_baths'] = str(increment)
    # add 1 to increment
    increment += 1
    
# for all data, make sure that beds_baths column is consistent 
counties_subset.loc[(counties_subset['beds_baths'] == "0.0") | (counties_subset['beds_baths'] == "0"), "beds_baths"] = "studio"
counties_subset.loc[(counties_subset['beds_baths'] == "1.0"), "beds_baths"] = "1"
counties_subset.loc[(counties_subset['beds_baths'] == "2.0"), "beds_baths"] = "2"
counties_subset.loc[(counties_subset['beds_baths'] == "3.0"), "beds_baths"] = "3"
counties_subset.loc[(counties_subset['beds_baths'] == "4.0"), "beds_baths"] = "4"
    
# "huge 12 bedroom house" "massive 12 bedroom house"



# drop all non-valid values from dataframe
# in order to run EDA on beds_baths column
df = counties_subset
# drop bed_bath values that were not matched using the title column from analysis
mask = df['no match'] == False
df = df[mask]
# subset data so as to only include studio - 4 bedrooms
df = df.loc[(df['beds_baths'] == "studio") | (df['beds_baths'] == "1") | (df['beds_baths'] == "2") | (df['beds_baths'] == "3") | (df['beds_baths'] == "4")]



# define function that can deal with subset of values of 
# beds_baths columns and perform various operations such as standard deviation
# and mean
def beds_statistics(df, column_name, column_boolean, beds_number = False):
    # if beds_number parameter is empty, function runs through all beds values
    if beds_number is False:
        # variable to increment and set as dictionary key
        count_beds = 0
        # create dictionary to store values 
        values_dict = {}
        # bed values studio -4 exist in dataset
        for value in range(5):
            # account for "studio" value in beds_baths which is a non-int
            if value == 0:
                count_beds = "studio"
                subset = df.loc[(df[column_boolean] == "True") & (df["beds_baths"] == "studio")]
                
            else:
            # subset data by beds value through each run in loop
                subset = df.loc[(df[column_boolean] == "True") & (df["beds_baths"] == str(count_beds))]
            
            # make sure subset only includes non-null values of whatever attribute column is input into 
            # the function
            subset = subset.loc[subset[column_boolean] == "True"]
            
            # convert column of input attribute to numeric
            subset[column_name] = pd.to_numeric(subset[column_name], errors='coerce')
            # get mean of subset column
            mean_value = (subset[column_name].mean())
            mean_value = round(mean_value, 4)
            # get standard deviation of subset column
            std_value = np.std(subset[column_name])
            std_value = round(std_value, 5)
            
            #convert to array so as to use iqr function from statsmodel
            new_array = subset[column_name].dropna()
            new_array = new_array.as_matrix()
            # call iqr function
            iqr_value = stats.iqr(new_array, axis=None, rng=(25, 75), scale='raw', nan_policy='propagate', interpolation='linear', keepdims=False)
            new_array = subset[column_name].dropna()
            new_array = new_array.as_matrix()
            # get 25th and 75th percentiles of subsetted column 
            q1, q3 = np.percentile(new_array, [25, 75])
            # set lower boundary as 1.5 standard deviations below the 25th percentile as 
            # than that will produce negative values
            lower = q1 - 1.5*(iqr_value)
            # set upper boundary as 3 standard deviations above the 75th percentile so as to
            # get rid of impossible rent values
            upper = q3 + 3.0*(iqr_value)
            # range of acceptable values will be those within the lower and upper bounds
            outlier_range = (lower, upper)
            # add percentiles to output dictionary
            percentiles = (q1, q3)
            # create dictionary key and nested dictionary as value
            values_dict[count_beds] = {'iqr': iqr_value, 'mean': mean_value, 'std': std_value, 'outlier range': outlier_range, "percentiles": percentiles}
            # account for "studio" value in beds_baths column
            if count_beds == "studio":
                count_beds = 0
             # increment count of beds values so as to perform operations on next value from beds column
            count_beds += 1
        return values_dict
   
    else:
        # create dictionary to store values 
        values_dict = {}
        # subset data by bed value entered in as beds_number parameter
        mask = df['beds_baths'] == float(beds_number)
        subset = df[mask]
        # make sure subset only includes non-null values of whatever attribute column is input into 
        # the function
        subset = subset.loc[subset[column_boolean] == "True"]
        
        # convert column of input attribute to numeric
        subset[column_name] = pd.to_numeric(subset[column_name], errors='coerce')
        # get mean of subset column
        mean_value = (subset[column_name].mean())
        mean_value = round(mean_value, 2)
        # get standard deviation of subset column
        std_value = np.std(subset[column_name])
        std_value = round(std_value, 5)
        subset = subset.dropna()
        #conver to array so as to use iqr function from statsmodel
        new_array = subset[column_name].dropna()
        
        # drop na from new array
        #new_array.dropna()
        # call iqr function
        iqr_value = stats.iqr(new_array, axis=None, rng=(25, 75), scale='raw', nan_policy='propagate', interpolation='linear', keepdims=False)
        # get 25th and 75th percentiles of subsetted column 
        q1, q3 = np.percentile(new_array, [25, 75])
        # set lower boundary as 1.5 standard deviations below the 25th percentile as 
        # than that will produce negative values
        lower = q1 - 1.5*(iqr_value)
        # set upper boundary as 3 standard deviations above the 75th percentile so as to
        # get rid of impossible rent values
        upper = q3 + 3.0*(iqr_value)
        # range of acceptable values will be those within the lower and upper bounds
        outlier_range = (lower, upper)
        # add percentiles to output dictionary
        percentiles = (q1, q3)


        # create dictionary key and nested dictionary as value
        values_dict[count_beds] = {'iqr': iqr_value, 'mean': mean_value, 'std': std_value, 'outlier range': outlier_range, "percentiles": percentiles}
     
        return values_dict

# create dictionaries of descriptive statistics values for price column 
# subsetted by bed value
price_dict = beds_statistics(df, "price", "price boolean", beds_number = False)

# create dictionaries of descriptive statistics values for square feet column 
# subsetted by bed value
sqft_dict = beds_statistics(df, "square feet", "square feet boolean", beds_number = False)

# function to identify outliers in the data using previously defined outlier range
def find_outliers(df, column_boolean, column_name, dictionary):
    # subset only non-null values of whatever attribute column is input into the 
    # function and conver to numeric
    df = df.loc[df[column_boolean] == "True"]
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
    # Set column to identify outliers
    df['outlier'] = False
    # loop through bed values keys in dictionaries and determine whether values are outliers
    for key in dictionary.items():
        df.loc[(df["beds_baths"] == str(key[0])) & (df[column_name] < (key[1]['outlier range'][0])), "outlier"] = True
        df.loc[(df["beds_baths"] == str(key[0])) & (df[column_name] > (key[1]['outlier range'][1])), "outlier"] = True
    return df
        
    

# create dataframe that identifies outliers using function
prices_outliers = find_outliers(df, "price boolean", "price", price_dict)



# set an index of integers equal to the length of dataframe rows
index_list = list(range(len(prices_outliers.index)))
# make index series a column in dataframe
prices_outliers['index'] = index_list
# set index column as index of dataframe
prices_outliers = prices_outliers.set_index(['index'])


# subset dataframe to only contain non-outliers
prices_no_outliers = prices_outliers.loc[prices_outliers['outlier'] == False]


# create groupby object of outliers to see where they tend to be distributed among beds groupings
outliers_grouped = prices_outliers.groupby(['beds_baths']).agg({'outlier' : 'sum' })

print ("outliers grouped", outliers_grouped)

# create groupby objects to see where the proportion of outliers are distributed
outliers_props = prices_outliers.groupby('beds_baths')["outlier"].value_counts()
outliers_percents = outliers_props.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
print ("outlier proportions", outliers_props)
print ("outlier percents", outliers_percents)

# get the statistics for the price column of dataframe without outliers
prices_no_outliers_statistics = beds_statistics(prices_no_outliers, "price", "price boolean", beds_number = False)
# do the same for square feet, but using prices_no_outliers dataframe
sqft_no_outliers_statistics = beds_statistics(prices_no_outliers, "square feet", "square feet boolean", beds_number = False)

# using prices_no_outliers_statistics dictionary
# create dictionary of standard deviation and mean of 
# prices and square feet using the per beds grouping
# for future use in data analysis with fair market rents
price_mean_dict = {}
price_std_dict = {}
sqft_mean_dict = {}
sqft_std_dict = {}
for value in prices_no_outliers_statistics.items():
    price_mean_dict[str(value[0])] = value[1]['mean']
    price_std_dict[str(value[0])] = value[1]['std']
for value in sqft_no_outliers_statistics.items():
    sqft_mean_dict[value[0]] = value[1]['mean']
    sqft_std_dict[value[0]] = value[1]['std']

# fill null values with mean from prices_no_outliers_statistics dictionary

# declare final dataframe
final_df = prices_no_outliers


# make sure that price and square feeet values are null where 'price boolean'
# indicates False to be able to fill them with mean values
final_df.loc[final_df['price boolean'] == "False", "price"] = np.nan
final_df.loc[final_df['square feet boolean'] == "False", "square feet"] = np.nan

# fill null values with the mean values from 
final_df['price'] = final_df['price'].fillna(df['beds_baths'].apply(lambda x: price_mean_dict.get(x)))
final_df['square feet'] = final_df['square feet'].fillna(df['beds_baths'].apply(lambda x: sqft_mean_dict.get(x)))

# drop the index_left column to avoid future problems
final_df = final_df.drop(columns = ['index_left'])
final_df.to_csv('cleaned_data_final.csv')



C:\Anaconda\lib\site-packages\ipykernel_launcher.py:256: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[ite

outliers grouped             outlier
beds_baths         
1              44.0
2             213.0
3               9.0
4               1.0
studio         86.0
outlier proportions beds_baths  outlier
1           False      9045
            True         44
2           False      7999
            True        213
3           False      1724
            True          9
4           False       513
            True          1
studio      False      1441
            True         86
Name: outlier, dtype: int64
outlier percents beds_baths  outlier
1           False      99.515898
            True        0.484102
2           False      97.406235
            True        2.593765
3           False      99.480669
            True        0.519331
4           False      99.805447
            True        0.194553
studio      False      94.368042
            True        5.631958
Name: outlier, dtype: float64


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:328: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
